- https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/chatbot-simulation-evaluation/agent-simulation-evaluation.ipynb
- 左右互博，角色扮演；

In [3]:
from dotenv import load_dotenv
assert load_dotenv()

In [24]:
from typing import List
import openai
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.adapters.openai import convert_message_to_dict

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

In [4]:
# This is flexible, but you can define your agent here, or call your agent API here.
def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    messages = [system_message] + messages
    completion = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo"
    )
    return completion.choices[0].message.model_dump()

In [5]:
my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': "Hello! Welcome to our airline's customer support. How can I assist you today?",
 'refusal': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None}

#### Define Simulated User

In [6]:
system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [7]:
prompt.pretty_print()

================================ System Message ================================

You are a customer of an airline company. You are interacting with a user who is a customer support person. 
{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'

============================= Messages Placeholder =============================

{messages}


In [10]:
instructions = """Your name is Harrison. You are trying to get a refund for the trip you took to Alaska. \
You want them to give you ALL the money back. \
This trip happened 5 years ago."""

prompt = prompt.partial(instructions=instructions)

prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [11]:
model = ChatOpenAI(model='gpt-4o-2024-08-06')
simulated_user = prompt | model

In [16]:
from rich.pretty import pprint
pprint(prompt.invoke({'messages': [HumanMessage(content='Hi! How can I help you?')]}))

ChatPromptValue(
│   messages=[
│   │   SystemMessage(
│   │   │   content="You are a customer of an airline company. You are interacting with a user who is a customer support person. \nYour name is Harrison. You are trying to get a refund for the trip you took to Alaska. You want them to give you ALL the money back. This trip happened 5 years ago.\n\nWhen you are finished with the conversation, respond with a single word 'FINISHED'",
│   │   │   additional_kwargs={},
│   │   │   response_metadata={}
│   │   ),
│   │   HumanMessage(content='Hi! How can I help you?', additional_kwargs={}, response_metadata={})
│   ]
)

In [17]:
simulated_user.invoke({'messages': [HumanMessage(content='Hi! How can I help you?')]})

AIMessage(content="Hi, I'm looking to get a refund for a trip I took to Alaska a while back. I believe I should be entitled to a full refund. Can you assist me with this?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 98, 'total_tokens': 135, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'stop', 'logprobs': None}, id='run-da7a524b-2e50-4e26-bf04-71dd26b8540c-0', usage_metadata={'input_tokens': 98, 'output_tokens': 37, 'total_tokens': 135})

#### Define the Agent Simulation

In [20]:
def chat_bot_node(state):
    messages = state["messages"]
    # Convert from LangChain format to the OpenAI format, which our chatbot function expects.
    messages = [convert_message_to_dict(m) for m in messages]
    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    # Respond with an AI Message
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

In [21]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages

In [22]:
def simulated_user_node(state):
    messages = state["messages"]
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    # This response is an AI message - we need to flip this to be a human message
    return {"messages": [HumanMessage(content=response.content)]}

#### define the edges

In [23]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

#### graph

In [25]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

In [26]:
for chunk in simulation.stream({"messages": []}):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': {'messages': [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}, id='1102460a-3f29-42ab-86ed-36bbe75404e3')]}}
----
{'user': {'messages': [HumanMessage(content="Hi, I'm looking to get a refund for a trip I took to Alaska. Could you help me with that?", additional_kwargs={}, response_metadata={}, id='3642dbba-06f6-462c-9657-02b1502029b2')]}}
----
{'chat_bot': {'messages': [AIMessage(content="I'd be happy to help you with that. Can you please provide me with your booking reference or ticket number so I can look up your reservation?", additional_kwargs={}, response_metadata={}, id='9be2b7ad-a3e1-41fe-86d7-303828262446')]}}
----
{'user': {'messages': [HumanMessage(content="I'm afraid I don't have the booking reference or ticket number anymore. The trip was about 5 years ago. Can you still help me with the refund?", additional_kwargs={}, response_metadata={}, id='13b11570-259e-4510-9b11-ae2f62aa6604')]}}
----
{'chat_bot': {'messa